# Capstone Project - The Battle of the Neighborhoods (Week 2)

### [1. Introduction: Business problem.](#1)

### [2. Data.](#2)

* [2.1. Importing python libraries.](#2.1)

* [2.2. Reading data on European capitals.](#2.2)
    * [2.2.1. Getting data from the wiki page.](#2.2.1)
    * [2.2.2. Parsing contents of wiki page.](#2.2.2)
    * [2.2.3. Processing and formatting data.](#2.2.3)

* [2.3. Getting geospatial data on European capitals.](#2.3)

* [2.4. Reading data on European capitals venues.](#2.4)
    * [2.4.1. Getting data over Foursquare REST API.](#2.4.1)
    * [2.4.2. Grouping venues by Capital and Venue Category.](#2.4.2)

### [3. Methodology](#3)

* [3.1. Visualization of European capitals on a world map.](#3.1)
* [3.2. What and how many top 10 most common venues do European capitals have?](#3.2)
* [3.3. Clustering data by Capitals similarity on venues.](#3.3)
    * [3.3.1. Scaling data.](#3.3.1)
    * [3.3.2. Fitting model.](#3.3.2)

### [4. Results](#4)

* [4.1. Merge cluster labels with top venues by Capital.](#4.1)
* [4.2. Merge cluster labels with geospatial data by Capital.](#4.2)
* [4.3. Visualizing European Capitals with colorize of Clusters.](#4.3)

### [5. Discussion](#5)

### [6. Conclusion.](#6)

# 1. Introduction: Business problem.<a class="anchor" id="1"></a>

### The main question is to understand how European capitals look alike in the development of their infrastructure.

# 2. Data.<a class="anchor" id="2"></a>

### 2.1. Importing python libraries.<a class="anchor" id="2.1"></a>

In [1]:
import numpy as np
import pandas as pd
import requests
import folium
import bs4
import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### 2.2. Reading data on European capitals.<a class="anchor" id="2.2"></a>

#### 2.2.1. Getting data on European capitals from the wiki page.<a class="anchor" id="2.2.1"></a>

In [2]:
# read web page
res = requests.get("https://en.wikipedia.org/wiki/Europe#List_of_states_and_territories")

#### 2.2.2. Parsing contents of wiki page.<a class="anchor" id="2.2.2"></a>

In [3]:
# function to parse table tag
def get_tag_name(tag):
    if tag.a == None:
        ret = tag.string
    elif tag.a.string == None:
        ret = " ".join([ str(i) for i in tag.a.contents if isinstance(i, bs4.element.NavigableString)])
    else:
        ret = tag.a.string
    return str(ret).strip()

In [4]:
# parse web page
soup = BeautifulSoup(res.text, 'html.parser')

In [5]:
# parse interested table
h2 = soup.find(lambda tag: tag.name=='h2' and tag.string=="List of states and territories")

head = []
rows = []
# parse tables
for tab in h2.find_next_siblings(name="table"):
    # parse rows
    for tr in tab.find_all(lambda tag: tag.name=='tr'):
        row = []
        if head == []:
            # parse header elements
            for th in tr.find_all(lambda tag: tag.name=='th'):
                head.append(get_tag_name(th))
        else:
            # parse row elements
            for td in tr.find_all(lambda tag: tag.name=='td'):
                row.append(get_tag_name(td))
            rows.append(row)
    # read the first table only 
    break

# create pandas dataframe
df = pd.DataFrame(data=rows, columns=head)
df.info()

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Flag                             51 non-null     object
 1   Arms                             51 non-null     object
 2   Name                             51 non-null     object
 3   Area                             51 non-null     object
 4   Population                       51 non-null     object
 5   Population density               51 non-null     object
 6   Capital                          51 non-null     object
 7   Name(s) in official language(s)  50 non-null     object
dtypes: object(8)
memory usage: 3.3+ KB


,Flag,Arms,Name,Area,Population,Population density,Capital,Name(s) in official language(s)
0,,,Albania,"28,748","2,876,591",98.5,Tirana,Shqipëria
1,,,Andorra,468,"77,281",179.8,Andorra la Vella,Andorra
2,,,Armenia,"29,743","2,924,816",101.5,Yerevan,Հայաստան (Hayastan)
3,,,Austria,"83,858","8,823,054",104,Vienna,Österreich
4,,,Azerbaijan,"86,600","9,911,646",113,Baku,Azǝrbaycan


#### 2.2.3. Processing and formatting data.<a class="anchor" id="2.2.3"></a>

In [6]:
eu_capitals = df.copy()

# drop the row Total
eu_capitals.drop(index=eu_capitals[ eu_capitals["Flag"] == "Total" ].index, inplace=True)

# drop the colums Flag, Arms, Name (s) in official language(s)
eu_capitals.drop(columns=["Flag",
                          "Arms",
                          "Population density",
                          "Name(s) in official language(s)"], inplace=True)

# remove comma in the number columns
for column in ["Area", "Population"]:
    eu_capitals[column] = eu_capitals.apply(lambda x: x[column].replace(",", ""), axis=1)

# convert number columns to appropriated types
eu_capitals = eu_capitals.astype({"Area":np.float64,
                                  "Population":np.int64})

# rename columns
eu_capitals.rename(columns={"Name":"Country"}, inplace=True)

eu_capitals.info()
eu_capitals.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Country     50 non-null     object 
 1   Area        50 non-null     float64
 2   Population  50 non-null     int64  
 3   Capital     50 non-null     object 
dtypes: float64(1), int64(1), object(2)
memory usage: 2.0+ KB


,Country,Area,Population,Capital
0,Albania,28748.0,2876591,Tirana
1,Andorra,468.0,77281,Andorra la Vella
2,Armenia,29743.0,2924816,Yerevan
3,Austria,83858.0,8823054,Vienna
4,Azerbaijan,86600.0,9911646,Baku


### 2.3. Reading geospatial data on European capitals.<a class="anchor" id="2.3"></a>

In [7]:
geolocator = Nominatim(user_agent="Mozilla/5.0")

lat = []
lng = []
# get geospatial coordinates for all european countries
for country, capital in zip(eu_capitals["Country"], eu_capitals["Capital"]):
    location = geolocator.geocode(country + ", " + capital)
    lat.append(location.latitude)
    lng.append(location.longitude)

# save results into dataset
eu_capitals["Latitude"] = lat
eu_capitals["Longitude"] = lng

# check results
eu_capitals.info()
eu_capitals

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Country     50 non-null     object 
 1   Area        50 non-null     float64
 2   Population  50 non-null     int64  
 3   Capital     50 non-null     object 
 4   Latitude    50 non-null     float64
 5   Longitude   50 non-null     float64
dtypes: float64(3), int64(1), object(2)
memory usage: 2.7+ KB


,Country,Area,Population,Capital,Latitude,Longitude
0,Albania,28748.00,2876591,Tirana,41.326873,19.818791
1,Andorra,468.00,77281,Andorra la Vella,42.506939,1.521247
2,Armenia,29743.00,2924816,Yerevan,40.177612,44.512585
3,Austria,83858.00,8823054,Vienna,48.208354,16.372504
4,Azerbaijan,86600.00,9911646,Baku,40.375443,49.832675
5,Belarus,207560.00,9504700,Minsk,53.902334,27.561879
6,Belgium,30528.00,11358357,Brussels,50.846557,4.351697
7,Bosnia and Herzegovina,51129.00,3531159,Sarajevo,43.851977,18.386687
8,Bulgaria,110910.00,7101859,Sofia,42.697863,23.322179
9,Croatia,56542.00,4284889,Zagreb,45.813177,15.977048


### 2.4. Reading data on European capitals venues.<a class="anchor" id="2.4"></a>

In [8]:
CLIENT_ID     = 'paste-your-client-id' # your Foursquare ID
CLIENT_SECRET = 'paste-your-client-secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT   = 1000       # limit of number of venues returned by Foursquare API
RADIUS  = 5000       # radius, meters

#### 2.4.1. Getting data over Foursquare REST API.<a class="anchor" id="2.4.1"></a>

In [9]:
venues_df = pd.DataFrame(columns=['Capital','Venue Category'])

for capital, lat, lng in zip(eu_capitals['Capital'],
                             eu_capitals['Latitude'],
                             eu_capitals['Longitude']):

    url = 'https://api.foursquare.com/v2/venues/explore' + \
          '?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}' \
          .format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, RADIUS, LIMIT)

    # perform request
    result = requests.get(url).json()
    if result["meta"]["code"] != 200:
        print("%s %d: %s" % \
              (result["meta"]["errorType"].upper(),
               result["meta"]["code"],
               result["meta"]["errorDetail"]))
        break

    # convert json into pandas.DataFrame
    venues = result["response"]["groups"][0]["items"]
    venues = pd.json_normalize(venues)

    # append data to results dataframe
    venues_ls = [ i[0]["name"] for i in venues["venue.categories"] ]
    venues_df = venues_df.append(other=pd.DataFrame(zip([ capital for i in range(len(venues_ls)) ],
                                                        venues_ls),
                                                    columns=("Capital", "Venue Category")))

venues_df.info()
venues_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4873 entries, 0 to 99
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Capital         4873 non-null   object
 1   Venue Category  4873 non-null   object
dtypes: object(2)
memory usage: 114.2+ KB


,Capital,Venue Category
0,Tirana,Hotel
1,Tirana,Italian Restaurant
2,Tirana,French Restaurant
3,Tirana,Lounge
4,Tirana,Ice Cream Shop


#### 2.4.2. Grouping venues by Capital and Venue Category.<a class="anchor" id="2.4.2"></a>

Check dataframe statistics.

In [10]:
venues_df.describe(include='all')

,Capital,Venue Category
count,4873,4873
unique,50,351
top,Amsterdam,Hotel
freq,100,282


After grouping we should have shape (50, 352) because (Capitals, 1 for Capital + Venue Categories).

In [11]:
venues_gby = pd.DataFrame({'Count' : venues_df.groupby( [ "Capital", "Venue Category"] ).size()}).reset_index()

# transpose (encode to onehot vectors) grouped data by Capital
venues_gby = venues_gby.pivot(index="Capital", columns="Venue Category", values="Count").fillna(0).reset_index()
venues_gby.columns.name = ""
venues_gby.shape

(50, 352)

Check data.

In [12]:
venues_gby

,Capital,ATM,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Bathing Area,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Bulgarian Restaurant,Burger Joint,Burrito Place,Butcher,Cable Car,Cafeteria,Café,Camera Store,Campground,Canal,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Chaat Place,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Library,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dutch Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kafenio,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Kumpir Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Magirio,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meyhane,Meze Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monastery,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Event,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paella Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pharmacy,Pie Shop,Pier,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Radio Station,Ramen Restaurant,Record Shop,Recreation Center,Resort,Rest Area,Restaurant,River,Road,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Science Museum,Scottish Restaurant,Sculpture Garden,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chairlift,Ski Lodge,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Clu

# 3. Methodology<a class="anchor" id="3"></a>

1. We have collected data about city venues in 5 kilometers around centre point of european cities.

2. After that we grouped this data for all venue categories by capital.

3. We will scaled amount of venue categories before training model.

4. We will use k-means clustering algorithm for training model.

### 3.1. Visualization of European capitals on a world map.<a class="anchor" id="3.1"></a>

In [13]:
#find centre of map
lat_centre = eu_capitals['Latitude'].mean()
lng_centre = eu_capitals['Longitude'].mean()

print("Centre point of map: %f, %f" % (lat_centre, lng_centre))

Centre point of map: 47.713142, 18.011288


In [14]:
# create map of Europe using latitude and longitude values
eu_map = folium.Map(location=[lat_centre, lng_centre], zoom_start=3)

# add markers to map
for country, capital, lat, lng in zip(eu_capitals['Country'],
                                      eu_capitals['Capital'],
                                      eu_capitals['Latitude'],
                                      eu_capitals['Longitude']):
    label = '{}, {}'.format(capital, country)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(eu_map)  
    
eu_map

### 3.2. What and how many top 10 most common venues do European capitals have?<a class="anchor" id="3.2"></a>

In [15]:
num_top_venues = 10

suffix = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Capital']
for i in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(i + 1, suffix[i]))
    except:
        columns.append('{}th Most Common Venue'.format(i + 1))

# create a new dataframe
venues_oby = pd.DataFrame(columns=columns)
venues_oby['Capital'] = venues_gby['Capital']

# fill dataframe with top data values
for i in np.arange(venues_gby.shape[0]):
    oby_dsc = venues_gby.iloc[i, 1:].sort_values(ascending=False)
    for j, cnt, idx in zip(range(num_top_venues),
                           oby_dsc.values[0:num_top_venues],
                           oby_dsc.index.values[0:num_top_venues]):
        venues_oby.iloc[i, j+1] = "{} {}".format(int(cnt), idx)

venues_oby

,Capital,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amsterdam,10 Coffee Shop,7 Hotel,4 Bar,4 Café,3 Plaza,3 Bookstore,3 French Restaurant,2 Bakery,2 Salad Place,2 Beer Bar
1,Andorra la Vella,17 Hotel,15 Restaurant,5 Spanish Restaurant,4 Café,3 Pizza Place,3 Resort,3 Diner,3 Coffee Shop,3 Bakery,2 Gym / Fitness Center
2,Ankara,7 Theater,7 History Museum,7 Café,5 Coffee Shop,4 Restaurant,3 Art Gallery,3 Bookstore,3 Meyhane,3 Bar,3 Dance Studio
3,Athens,9 Café,9 Historic Site,9 Bar,9 Coffee Shop,5 Dessert Shop,4 Cocktail Bar,3 Ice Cream Shop,3 Meze Restaurant,3 Falafel Restaurant,2 Theater
4,Baku,17 Park,9 Hotel,9 Coffee Shop,6 Café,4 Plaza,3 Historic Site,3 Restaurant,3 Lounge,2 Kebab Restaurant,2 Spa
5,Belgrade,6 Bar,5 Hotel,4 Cocktail Bar,4 Ice Cream Shop,4 Park,3 Eastern European Restaurant,3 Theater,3 Pub,3 Restaurant,3 Plaza
6,Berlin,10 Coffee Shop,7 Park,6 Bookstore,4 Concert Hall,4 Ice Cream Shop,3 Hotel,3 Sandwich Place,3 Wine Bar,3 Bakery,3 Monument / Landmark
7,Bern,8 Café,7 Swiss Restaurant,7 Bar,6 Plaza,6 Park,4 Restaurant,3 Shopping Mall,3 Ice Cream Shop,2 Coffee Shop,2 Pizza Place
8,Bratislava,12 Café,10 Coffee Shop,3 Bar,3 Vegetarian / Vegan Restaurant,3 Pizza Place,3 Ice Cream Shop,3 Italian Restaurant,3 Beer Bar,2 Plaza,2 Creperie
9,Brussels,11 Bar,5 Concert Hall,5 Plaza,5 Coffee Shop,5 Sandwich Place,5 Bookstore,4 Chocolate Shop,3 Park,3 Brewery,3 Belgian Restaurant


### 3.3. Clustering data by Capitals similarity on venues.<a class="anchor" id="3.3"></a>

#### 3.3.1. Scaling data.<a class="anchor" id="3.3.1"></a>

In [16]:
# scale features
scaler = StandardScaler()
eu_capitals_scaled = scaler.fit_transform(venues_gby.drop('Capital', 1))

Checking shape scaled features.

In [17]:
print(eu_capitals_scaled.shape)

(50, 351)


Checking first scaled features.

In [18]:
print(eu_capitals_scaled[0:5])

[[-0.14285714 -0.14285714 -0.14285714 ... -0.14285714  2.21057185
   2.7080128 ]
 [-0.14285714 -0.14285714 -0.14285714 ... -0.14285714 -0.48524748
  -0.36927447]
 [-0.14285714 -0.14285714 -0.14285714 ...  7.          2.21057185
  -0.36927447]
 [-0.14285714 -0.14285714 -0.14285714 ... -0.14285714 -0.48524748
  -0.36927447]
 [-0.14285714 -0.14285714 -0.14285714 ... -0.14285714 -0.48524748
  -0.36927447]]


#### 3.3.2. Fitting model.<a class="anchor" id="3.3.2"></a>

In [19]:
# set number of clusters
kclusters = 10

# run k-means clustering
clustering = KMeans(n_clusters=kclusters, random_state=1).fit(eu_capitals_scaled)

# check cluster labels generated for each row in the dataframe
clustering.labels_

array([2, 3, 0, 8, 2, 2, 2, 5, 2, 2, 2, 2, 5, 7, 2, 2, 4, 2, 2, 2, 1, 2,
       2, 5, 6, 2, 5, 2, 2, 5, 2, 2, 5, 2, 2, 2, 5, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 9, 5, 5])

# 4. Results.<a class="anchor" id="4"></a>

### 4.1. Merge cluster labels with top venues by Capital.<a class="anchor" id="4.1"></a>

In [20]:
eu_capitals_clustered = venues_oby.copy()
# add clustering labels
eu_capitals_clustered.insert(0, 'Cluster', clustering.labels_)
# resort data by Cluster and Capital
eu_capitals_clustered.sort_values(by=["Cluster", "Capital"], inplace=True)
eu_capitals_clustered.reset_index(drop=True, inplace=True)
eu_capitals_clustered

,Cluster,Capital,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Ankara,7 Theater,7 History Museum,7 Café,5 Coffee Shop,4 Restaurant,3 Art Gallery,3 Bookstore,3 Meyhane,3 Bar,3 Dance Studio
1,1,Luxembourg,8 Bar,6 Italian Restaurant,5 Hotel,5 French Restaurant,5 Indian Restaurant,5 Coffee Shop,4 Café,4 Park,3 Bakery,3 Restaurant
2,2,Amsterdam,10 Coffee Shop,7 Hotel,4 Bar,4 Café,3 Plaza,3 Bookstore,3 French Restaurant,2 Bakery,2 Salad Place,2 Beer Bar
3,2,Baku,17 Park,9 Hotel,9 Coffee Shop,6 Café,4 Plaza,3 Historic Site,3 Restaurant,3 Lounge,2 Kebab Restaurant,2 Spa
4,2,Belgrade,6 Bar,5 Hotel,4 Cocktail Bar,4 Ice Cream Shop,4 Park,3 Eastern European Restaurant,3 Theater,3 Pub,3 Restaurant,3 Plaza
5,2,Berlin,10 Coffee Shop,7 Park,6 Bookstore,4 Concert Hall,4 Ice Cream Shop,3 Hotel,3 Sandwich Place,3 Wine Bar,3 Bakery,3 Monument / Landmark
6,2,Bratislava,12 Café,10 Coffee Shop,3 Bar,3 Vegetarian / Vegan Restaurant,3 Pizza Place,3 Ice Cream Shop,3 Italian Restaurant,3 Beer Bar,2 Plaza,2 Creperie
7,2,Brussels,11 Bar,5 Concert Hall,5 Plaza,5 Coffee Shop,5 Sandwich Place,5 Bookstore,4 Chocolate Shop,3 Park,3 Brewery,3 Belgian Restaurant
8,2,Bucharest,17 Coffee Shop,7 Hotel,7 Dessert Shop,4 Burger Joint,4 Park,4 Plaza,3 Italian Restaurant,3 Romanian Restaurant,2 Pizza Place,2 Bookstore
9,2,Budapest,11 Coffee Shop,9 Hotel,4 Bakery,4 Pizza Place,3 Dessert Shop,3 Beer Bar,3 Ice Cream Shop,3 Italian Restaurant,2 Outdoor Sculpture,2 Gourmet Shop


### 4.2. Merge cluster labels with geospatial data by Capital.<a class="anchor" id="4.2"></a>

In [21]:
eu_capitals_summary = eu_capitals_clustered[["Cluster","Capital"]].join(eu_capitals.set_index('Capital'), on='Capital')
eu_capitals_summary

,Cluster,Capital,Country,Area,Population,Latitude,Longitude
0,0,Ankara,Turkey,783356.00,80810525,39.920777,32.854067
1,1,Luxembourg,Luxembourg,2586.00,602005,49.611277,6.129799
2,2,Amsterdam,Netherlands,41543.00,17271990,52.372760,4.893604
3,2,Baku,Azerbaijan,86600.00,9911646,40.375443,49.832675
4,2,Belgrade,Serbia,88361.00,7040272,44.817813,20.456897
5,2,Berlin,Germany,357168.00,82800000,52.517037,13.388860
6,2,Bratislava,Slovakia,49035.00,5435343,48.151699,17.109306
7,2,Brussels,Belgium,30528.00,11358357,50.846557,4.351697
8,2,Bucharest,Romania,238397.00,19638000,44.436141,26.102720
9,2,Budapest,Hungary,93030.00,9797561,47.498382,19.040471


### 4.3. Visualizing European Capitals with colorize of Clusters.<a class="anchor" id="4.3"></a>

In [22]:
# create map of Europe using latitude and longitude values
eu_map = folium.Map(location=[lat_centre, lng_centre], zoom_start=3)

# set color scheme for the clusters
rainbow = [colors.rgb2hex(i) for i in cm.rainbow(np.linspace(start=0, stop=1, num=kclusters, endpoint=True))]

# add markers to map
for cluster, country, capital, lat, lng in zip(eu_capitals_summary['Cluster'],
                                               eu_capitals_summary['Country'],
                                               eu_capitals_summary['Capital'],
                                               eu_capitals_summary['Latitude'],
                                               eu_capitals_summary['Longitude']):
    label = '{}: {}, {}'.format(cluster, capital, country)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color=rainbow[cluster-1],
                        fill=True,
                        fill_color=rainbow[cluster-1],
                        fill_opacity=0.7,
                        parse_html=False).add_to(eu_map)  
    
eu_map

# 5. Discussion.<a class="anchor" id="5"></a>

#### We see that some Capitals fall into separate clusters in which there is no one but themselves.

In [23]:
all_clusters = pd.DataFrame({'Capitals' : eu_capitals_summary.groupby( [ "Cluster" ] ).size()}).reset_index()
all_clusters

,Cluster,Capitals
0,0,1
1,1,1
2,2,33
3,3,1
4,4,1
5,5,9
6,6,1
7,7,1
8,8,1
9,9,1


#### What European capitals are not similar to other capitals in the development of their public infrastructure?

In [24]:
rare_clusters = all_clusters[ all_clusters["Capitals"] == 1 ]

rare_clusters = rare_clusters[["Cluster"]].join(eu_capitals_summary.set_index('Cluster'), on='Cluster')
rare_clusters

,Cluster,Capital,Country,Area,Population,Latitude,Longitude
0,0,Ankara,Turkey,783356.0,80810525,39.920777,32.854067
1,1,Luxembourg,Luxembourg,2586.0,602005,49.611277,6.129799
3,3,Andorra la Vella,Andorra,468.0,77281,42.506939,1.521247
4,4,Kiev,Ukraine,603628.0,42418235,50.450034,30.524136
6,6,Moscow,Russia,17098246.0,144526636,55.750446,37.617494
7,7,Copenhagen,Denmark,43094.0,5748796,55.686724,12.570072
8,8,Athens,Greece,131957.0,10768477,37.983941,23.728305
9,9,Warsaw,Poland,312685.0,38422346,52.233717,21.071411


# 6. Conclusion.<a class="anchor" id="6"></a>

The purpose of this project was to understand whether the mutual territorial proximity of states affects the development of the infrastructure of their capitals so that cities become similar to each other.

The answer is yes. We have a large cluster (2), which includes 33 EU states (in the majority) with similar categories of objects.